# Домашняя работа №1

## Задание:
### Структура:  ['Очередь']
### Метод выделения памяти:  ['Куча']
### Список функций (дейстивий):  ['Добавление в конец', 'Удаление из начала', 'Заполнение случайными числами', 'Получение списка нечетных чисел', 'Подсчет количества чисел, оканчивающихся на 1', 'Получение списка нечетных чисел']

```ASM

format ELF64

; === Экспортируемые функции для C ===
public queue_init
public queue_push
public queue_pop
public queue_fill_random
public queue_print_odds
public queue_count_end1

; === Секция данных (глобальные переменные) ===
section '.data' writeable
    q_data dq 0
    q_head dq 0
    q_tail dq 0
    q_size dq 0
    q_cap dq 0
    newline db 10

; === Основной код ===
section '.text' executable

; --- Инициализация очереди (heap) ---
queue_init:
    mov rdi, 256

    mov rax, 9
    mov rsi, rdi
    mov rdx, 0x3
    mov r10, 0x22
    xor r8, r8
    xor r9, r9
    syscall

    mov [q_data], rax
    mov qword [q_head], 0
    mov qword [q_tail], 0
    mov qword [q_size], 0
    mov qword [q_cap], 32
    ret

; --- Добавить элемент в конец ---
queue_push:
    push rbx

    mov rax, [q_size]
    mov rbx, [q_cap]
    cmp rax, rbx
    jge .done

    mov rbx, [q_tail]
    mov rcx, [q_data]
    mov [rcx + rbx*8], rdi
    inc rbx
    mov rdx, [q_cap]
    cmp rbx, rdx
    jl .no_over

    xor rbx, rbx

    .no_over:
        mov [q_tail], rbx
        inc qword [q_size]
    .done:
        pop rbx
        ret

; --- Удалить из начала, вернуть значение ---
queue_pop:
    push rbx
    mov rax, [q_size]
    test rax, rax
    jz .fail

    mov rbx, [q_head]
    mov rcx, [q_data]
    mov rdx, [rcx + rbx*8]
    inc rbx
    mov rsi, [q_cap]
    cmp rbx, rsi
    jl .no_over2

    xor rbx, rbx

    .no_over2:
        mov [q_head], rbx
        dec qword [q_size]
        mov rax, rdx
        pop rbx
        ret
    .fail:
        xor rax, rax
        pop rbx
        ret

; --- Заполнить случайными числами [0, maxv-1], n штук ---
queue_fill_random:
    mov rcx, rdi
    mov r8, rsi

    .lp_fill:
        test rcx, rcx
        jz .done_fill

        push rcx
        push r8

        rdtsc
        xor rdx, rdx
        div r8
        mov rdi, rdx
        call queue_push


        pop r8
        pop rcx

        dec rcx
        jmp .lp_fill
    .done_fill:
        ret

; --- Вывести все НЕЧЕТНЫЕ числа через print_int ---
queue_print_odds:
    mov rbx, [q_head]
    mov rcx, [q_size]
    mov rdx, [q_cap]
    mov rsi, [q_data]

    test rsi, rsi
    jz .done_odd

    .lp_odd:
        test rcx, rcx
        jz .done_odd
        mov rax, [rsi + rbx*8]
        test rax, 1
        jz .skip_odd


        push rbx
        push rcx
        push rdx
        push rsi

        mov rdi, rax
        call print_int
        call print_newline


        pop rsi
        pop rdx
        pop rcx
        pop rbx

    .skip_odd:
        inc rbx
        cmp rbx, rdx
        jl .no_over_odd

        xor rbx, rbx
    .no_over_odd:
        dec rcx
        jmp .lp_odd
    .done_odd:
        ret

; --- Вернуть кол-во чисел в очереди, оканчивающихся на 1 ---
queue_count_end1:
    mov rbx, [q_head]
    mov rcx, [q_size]
    mov rdx, [q_cap]
    xor r8, r8

    mov rax, [q_data]

    test rax, rax
    jz .done_c1

    .lp_c1:
        test rcx, rcx
        jz .done_c1
        mov rax, [q_data]
        mov rsi, [rax + rbx*8]

        push rbx
        push rcx
        push rdx

        mov rax, rsi
        xor rdx, rdx
        mov r9, 10
        div r9
        cmp rdx, 1

        pop rdx
        pop rcx
        pop rbx

        jne .skip_c1
        inc r8
    .skip_c1:
        inc rbx
        cmp rbx, [q_cap]
        jl .no_over_c1
        xor rbx, rbx
    .no_over_c1:
        dec rcx
        jmp .lp_c1
    .done_c1:
        mov rax, r8
        ret

; ============ СЛУЖЕБНЫЕ ---------


print_int:
    push rbx
    push rbp
    mov rbp, rsp
    sub rsp, 32

    mov rcx, 10
    mov rax, rdi
    lea rbx, [rsp+31]
    mov byte [rbx], 0
    cmp rax, 0
    jge .positive

    neg rax
    mov bl, '-'
    dec rbx
    mov [rbx], bl

    .positive:
        mov r9, rbx
    .pi_1:
        xor rdx, rdx
        div rcx
        add dl, '0'
        dec rbx
        mov [rbx], dl
        test rax, rax
        jnz .pi_1


        mov rax, 1
        mov rdi, 1
        mov rsi, rbx
        mov rdx, r9
        sub rdx, rbx
        syscall

        mov rsp, rbp
        pop rbp
        pop rbx
        ret

; --- печать перевода строки ---
print_newline:
    mov rax, 1
    mov rdi, 1
    mov rsi, newline
    mov rdx, 1
    syscall
    ret

```

```C

#include <stdio.h>

extern void queue_init();
extern void queue_push(long val);
extern long queue_pop();
extern void queue_fill_random(int n, int maxv);
extern void queue_print_odds();
extern long queue_count_end1();

void print_menu() {
    printf("1. Добавить в конец\n");
    printf("2. Удалить из начала\n");
    printf("3. Заполнить случайными числами\n");
    printf("4. Получить список нечетных чисел\n");
    printf("5. Подсчитать количество чисел, оканчивающихся на 1\n");
    printf("0. Выйти\n");
}

int main() {
    queue_init();
    int cmd;
    while (1) {
        print_menu();
        printf("\nВаша команда: ");
        scanf("%d", &cmd);
        if (cmd == 0) break;
        if (cmd == 1) {
            long val;

            printf("Введите число: ");
            scanf("%ld", &val);


            queue_push(val);
        } else if (cmd == 2) {
            long x = queue_pop();

            printf("Удалено: %ld\n", x);
        } else if (cmd == 3) {
            int n, maxv;
            printf("Введите n и max_value: ");
            scanf("%d%d", &n, &maxv);

            queue_fill_random(n, maxv);
            printf("Заполнено!\n");
        } else if (cmd == 4) {
            queue_print_odds();
            printf("\n");
        } else if (cmd == 5) {
            long c = queue_count_end1();

            printf("Количество: %ld", c);
            printf("\n");
        }
    }
    return 0;
}
```